<a href="https://colab.research.google.com/github/nprentza/xaimedical/blob/main/notebooks/xgBoost_Anchors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/nprentza/xaimedical/main/datasets/diabetes_earlystage.csv'
df = pd.read_csv(url)

Separate features/predictors (x) and target variable (y).

In [2]:
x=df.drop(["patientclass"], axis=1)
y=df.patientclass

Split data into train and test.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

Train a XGBoost model and predict the response for the test set (x_test).

In [5]:
import xgboost as xgb

from sklearn.metrics import accuracy_score

xgc = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=42) # Creating a XGB Classifier
xgc.fit(X_train, y_train)

predictions = xgc.predict(X_test)

accuracy_score(y_test, predictions)

0.9615384615384616

# Explain predictions with ANCHORS

In [7]:
#!pip install anchor-exp
from anchor import utils
from anchor import anchor_tabular

In [8]:
features=["age","gender","polyuria","polydipsia","sudden_weight_loss","weakness","polyphagia","genital_thrush","visual_blurring","Itching","Irritability","delayed_healing","partial_paresis","muscle_stiffness","alopecia","obesity"]
target="patientclass"

explainer = anchor_tabular.AnchorTabularExplainer(['negative','positive'],features,X_train.values)

In [9]:
import numpy as np
idx = 0
np.random.seed(1)
prediction_index = xgc.predict(X_test.iloc[idx].values.reshape(1, -1))[0]
# Check if the prediction index is within the bounds of class_names
if 0 <= prediction_index < len(explainer.class_names):
    print('Prediction: ', explainer.class_names[prediction_index])
    exp = explainer.explain_instance(X_test.iloc[idx].values, xgc.predict, threshold=0.95)
else:
    print("Prediction index out of bounds. Check your model and class names.")

Prediction:  negative


In [10]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: gender <= 1.00 AND polydipsia <= 0.00 AND sudden_weight_loss <= 0.00 AND Irritability <= 0.00 AND polyphagia > 0.00
Precision: 0.96
Coverage: 0.08
